In [1]:
# =========================================================
# PyTorch + MLflow (Image Classification on MNIST)
# - Tracks to http://127.0.0.1:8080
# - Logs model with signature using mlflow.pytorch.log_model (returns model_info)
# - Tests with mlflow.pyfunc.load_model(model_info.model_uri)
# =========================================================

# If needed:
# !pip install torch torchvision mlflow numpy pandas

import os
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import mlflow
import mlflow.pytorch
from mlflow.models import infer_signature

# ---------- Config ----------
mlflow.set_tracking_uri("http://127.0.0.1:8081")
EXPERIMENT_NAME = "PyTorch_MNIST_Image_Demo"
mlflow.set_experiment(EXPERIMENT_NAME)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 128
EPOCHS = 1   # keep tiny for quick demo


/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
2025/10/06 15:33:05 INFO mlflow.tracking.fluent: Experiment with name 'PyTorch_MNIST_Image_Demo' does not exist. Creating a new experiment.


In [ ]:

# ---------- Data ----------
# Using simple ToTensor() so inputs are [0,1]; normalization is done in the model.
transform = transforms.ToTensor()
train_ds = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_ds  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

# ---------- Model ----------
class SmallCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # we do normalization inside the model to make pyfunc inference easier
        self.register_buffer("mean", torch.tensor([0.1307]).view(1,1,1,1))  # MNIST mean
        self.register_buffer("std",  torch.tensor([0.3081]).view(1,1,1,1))  # MNIST std

        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1   = nn.Linear(32*7*7, 64)
        self.fc2   = nn.Linear(64, num_classes)

    def forward(self, x):
        # x expected shape: [N, 1, 28, 28], float in [0,1]
        x = (x - self.mean) / self.std
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)          # 28 -> 14
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)          # 14 -> 7
        x = x.view(x.size(0), -1)       # [N, 32*7*7]
        x = F.relu(self.fc1(x))
        logits = self.fc2(x)
        return logits

model = SmallCNN().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# ---------- Train (quick) ----------
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            logits = model(xb)
            pred = logits.argmax(dim=1)
            correct += (pred == yb).sum().item()
            total += yb.numel()
    return correct / max(total, 1)



100%|██████████| 9.91M/9.91M [00:04<00:00, 2.26MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 48.9kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 836kB/s] 
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.36MB/s]


Epoch 1/1 - Val Acc: 0.9797


2025/10/06 15:33:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/06 15:33:27 WARNING mlflow.utils.requirements_utils: Found torch version (2.9.0a0+50eac811a6.nv25.9) contains a local version label (+50eac811a6.nv25.9). MLflow logged a pip requirement for this package as 'torch==2.9.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/10/06 15:33:31 WARNING mlflow.utils.requirements_utils: Found torch version (2.9.0a0+50eac811a6.nv25.9) contains a local version label (+50eac811a6.nv25.9). MLflow logged a pip requirement for this package as 'torch==2.9.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


Run ID: 6235bf533ece4528ade62f84680773cd
Model URI: models:/m-f1687339cf034944b50f115f389bb9bc
🏃 View run train-smallcnn-mnist at: http://127.0.0.1:8081/#/experiments/353988559714947429/runs/6235bf533ece4528ade62f84680773cd
🧪 View experiment at: http://127.0.0.1:8081/#/experiments/353988559714947429


PyFunc predict() -> probs shape: (8, 10)
Predicted labels: [7, 2, 1, 0, 4, 1, 4, 9]
True labels     : [7, 2, 1, 0, 4, 1, 4, 9]


In [ ]:
with mlflow.start_run(run_name="train-smallcnn-mnist") as run:
    mlflow.log_param("architecture", "SmallCNN")
    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("batch_size", BATCH_SIZE)
    mlflow.set_tag("task", "image-classification")
    mlflow.set_tag("dataset", "MNIST")

    for epoch in range(EPOCHS):
        model.train()
        for i, (xb, yb) in enumerate(train_loader):
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                mlflow.log_metric("train_loss", float(loss.item()), step=epoch*len(train_loader)+(i+1))

        acc = evaluate(model, test_loader)
        mlflow.log_metric("val_accuracy", float(acc), step=epoch)
        print(f"Epoch {epoch+1}/{EPOCHS} - Val Acc: {acc:.4f}")

    

In [4]:
# ---------- Build signature & input example ----------
# Use a small batch from test set (shape [N,1,28,28]) for signature
model.eval()
xb_example, _ = next(iter(test_loader))
xb_example = xb_example[:4].to(DEVICE)        # [4,1,28,28]
with torch.no_grad():
    yb_example = model(xb_example).softmax(dim=1).cpu().numpy()  # probs as output

input_example = xb_example.cpu().numpy()      # numpy [4,1,28,28]
signature = infer_signature(input_example, yb_example)

# ---------- Log model (returns model_info) ----------
# Saving as both PyTorch and PyFunc flavors. The PyFunc flavor will accept numpy or DataFrame.
model_info = mlflow.pytorch.log_model(
    pytorch_model=model,
    artifact_path="model",
    signature=signature,
    input_example=input_example,
    registered_model_name=None  # set a string here if you want to register the model
)

print("Run ID:", run.info.run_id)
print("Model URI:", model_info.model_uri)


2025/10/06 15:35:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/06 15:35:52 WARNING mlflow.utils.requirements_utils: Found torch version (2.9.0a0+50eac811a6.nv25.9) contains a local version label (+50eac811a6.nv25.9). MLflow logged a pip requirement for this package as 'torch==2.9.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/10/06 15:35:56 WARNING mlflow.utils.requirements_utils: Found torch version (2.9.0a0+50eac811a6.nv25.9) contains a local version label (+50eac811a6.nv25.9). MLflow logged a pip requirement for this package as 'torch==2.9.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


Run ID: 6235bf533ece4528ade62f84680773cd
Model URI: models:/m-855363ff7db549bf8378f96d32d78791


In [5]:

# ---------- Inference test with pyfunc ----------
# Load via pyfunc and run predict on a few images (expects array or DataFrame shaped [N,1,28,28])
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

# Prepare a tiny test batch as numpy
test_xb, test_yb = next(iter(test_loader))
test_x_np = test_xb[:8].numpy()  # [8,1,28,28]

# Option 1: numpy array input
probs_np = loaded_model.predict(test_x_np)  # returns numpy [8,10]
pred_labels = probs_np.argmax(axis=1)

print("PyFunc predict() -> probs shape:", probs_np.shape)
print("Predicted labels:", pred_labels.tolist())
print("True labels     :", test_yb[:8].numpy().tolist())

# Option 2: pandas DataFrame input (if you prefer)
# df_input = pd.DataFrame({"images": list(test_x_np)})  # not necessary here; numpy is simpler
# probs_from_df = loaded_model.predict(df_input)


PyFunc predict() -> probs shape: (8, 10)
Predicted labels: [7, 2, 1, 0, 4, 1, 4, 9]
True labels     : [7, 2, 1, 0, 4, 1, 4, 9]
